In [2]:
import pandas as pd
dataset=pd.read_csv("logs_dataset.csv")

In [3]:
dataset.head()

,@timestamp,_id,ip_address
0,"July 8th 2019, 14:43:03.000",XswJ0msBoTGddM7vxMDB,10.1.1.285
1,"July 8th 2019, 14:43:01.000",dKQJ0msB7mP0GwVzvJjz,10.1.2.389
2,"July 8th 2019, 14:42:59.000",CcwJ0msBoTGddM7vtb8y,10.1.1.415
3,"July 8th 2019, 14:42:57.000",bKQJ0msB7mP0GwVzrZdT,10.1.1.79
4,"July 8th 2019, 14:42:55.000",L6QJ0msB7mP0GwVzpZeI,10.1.1.60


In [4]:
len(dataset)

721547

In [5]:
dataset["_id"].unique()

array(['XswJ0msBoTGddM7vxMDB', 'dKQJ0msB7mP0GwVzvJjz',
       'CcwJ0msBoTGddM7vtb8y', ..., 'vUWJOWsBoTGddM7vHkGD',
       'aUWJOWsBoTGddM7vB0AR', 'AEWIOWsBoTGddM7v90Bw'],
      shape=(712431,), dtype=object)

In [8]:
dataset['@timestamp'] = (
    dataset['@timestamp']
    .str.replace(r'(\d+)(st|nd|rd|th)', r'\1', regex=True)
)
dataset['@timestamp'] = pd.to_datetime(
    dataset['@timestamp'],
    format='%B %d %Y, %H:%M:%S.%f'
)

In [9]:
dataset.sort_values(['ip_address', '@timestamp'], inplace=True)

In [13]:
dataset['shift_time'] = dataset.groupby(['ip_address'])['@timestamp'].shift(1)

In [14]:
dataset.head()

,@timestamp,_id,ip_address,shift_time
721473,2019-06-09 00:06:09,DBuOOWsB7mP0GwVzhZ9U,10.1.1.1,NaT
720483,2019-06-09 01:28:39,bB7aOWsB7mP0GwVzDY5G,10.1.1.1,2019-06-09 00:06:09
719233,2019-06-09 03:12:49,R0w5OmsBoTGddM7vayZT,10.1.1.1,2019-06-09 01:28:39
719222,2019-06-09 03:13:45,U0w6OmsBoTGddM7vRi8R,10.1.1.1,2019-06-09 03:12:49
718875,2019-06-09 03:42:39,z01UOmsBoTGddM7vuzyC,10.1.1.1,2019-06-09 03:13:45


In [15]:
dataset['time_diff'] = (dataset['@timestamp']-dataset['shift_time']).dt.seconds//60

In [16]:
dataset['date'] = dataset['@timestamp'].dt.date

In [18]:
dataset['weekday'] = dataset['@timestamp'].dt.weekday

In [19]:
dataset['hour'] = dataset['@timestamp'].dt.hour

In [20]:
dataset['is_weekend'] = ((dataset['weekday'] == 5) | (dataset['weekday'] == 6)).astype(int)

In [21]:
dataset

,@timestamp,_id,ip_address,shift_time,time_diff,date,weekday,hour,is_weekend
721473,2019-06-09 00:06:09,DBuOOWsB7mP0GwVzhZ9U,10.1.1.1,NaT,NaN,2019-06-09,6,0,1
720483,2019-06-09 01:28:39,bB7aOWsB7mP0GwVzDY5G,10.1.1.1,2019-06-09 00:06:09,82.0,2019-06-09,6,1,1
719233,2019-06-09 03:12:49,R0w5OmsBoTGddM7vayZT,10.1.1.1,2019-06-09 01:28:39,104.0,2019-06-09,6,3,1
719222,2019-06-09 03:13:45,U0w6OmsBoTGddM7vRi8R,10.1.1.1,2019-06-09 03:12:49,0.0,2019-06-09,6,3,1
718875,2019-06-09 03:42:39,z01UOmsBoTGddM7vuzyC,10.1.1.1,2019-06-09 03:13:45,28.0,2019-06-09,6,3,1
...,...,...,...,...,...,...,...,...,...
3225,2019-07-08 11:49:00,3cBq0WsBoTGddM7va5TJ,10.1.2.99,2019-07-08 11:47:15,1.0,2019-07-08,0,11,0
2422,2019-07-08 12:22:29,QMKJ0WsBoTGddM7vE9N1,10.1.2.99,2019-07-08 11:49:00,33.0,2019-07-08,0,12,0
1704,2019-07-08 12:52:25,9pyk0WsB7mP0GwVze7sV,10.1.2.99,2019-07-08 12:22:29,29.0,2019-07-08,0,12,0
1138,2019-07-08 13:15:59,B8a60WsBoTGddM7vDnqQ,10.1.2.99,2019-07-08 12:52:25,23.0,2019-07-08,0,13,0


In [22]:
dataset['hour_bucket'] = dataset['hour']//4
# This integer division by 4, so we can analyse 4-hour buckets instead of every single hour

In [24]:
ip_addr ='ip_address'

In [25]:
ip_counts = dataset.groupby(ip_addr)['@timestamp'].count().reset_index()

In [26]:
ip_counts.head()

,ip_address,@timestamp
0,10.1.1.1,1446
1,10.1.1.100,2860
2,10.1.1.101,1465
3,10.1.1.106,1408
4,10.1.1.109,1459


In [27]:
daily_counts = dataset.groupby([ip_addr,'date'])['@timestamp'].count().reset_index()

In [29]:
daily_counts

,ip_address,date,@timestamp
0,10.1.1.1,2019-06-09,36
1,10.1.1.1,2019-06-10,37
2,10.1.1.1,2019-06-11,70
3,10.1.1.1,2019-06-12,38
4,10.1.1.1,2019-06-13,32
...,...,...,...
11575,10.1.2.99,2019-07-04,79
11576,10.1.2.99,2019-07-05,61
11577,10.1.2.99,2019-07-06,89
11578,10.1.2.99,2019-07-07,47


In [30]:
daily_counts = daily_counts.rename(columns={'@timestamp':'daily_counts'})

In [31]:
daily_counts_avg = daily_counts.groupby(ip_addr).daily_counts.median().reset_index()

In [32]:
daily_counts_avg.head(5)

,ip_address,daily_counts
0,10.1.1.1,40.0
1,10.1.1.100,78.0
2,10.1.1.101,40.0
3,10.1.1.106,35.5
4,10.1.1.109,42.5


In [34]:
weekend_counts = dataset.groupby([ip_addr, 'is_weekend'])['@timestamp'].count().reset_index()

In [36]:
weekend_counts

,ip_address,is_weekend,@timestamp
0,10.1.1.1,0,975
1,10.1.1.1,1,471
2,10.1.1.100,0,1960
3,10.1.1.100,1,900
4,10.1.1.101,0,1006
...,...,...,...
767,10.1.2.90,1,871
768,10.1.2.95,0,1973
769,10.1.2.95,1,895
770,10.1.2.99,0,978
